In [20]:
import os
import pandas as pd 
import numpy as np
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [21]:
#加载数据集
url_feature =pd.read_csv('/Users/wuyijiong/Desktop/data/url_feature.csv', sep=',')
url_feature = url_feature.infer_objects()

In [22]:
#清洗数据
url_feature.set_index("id_date",inplace=True)
url_feature = url_feature.drop(["buyer_id","date"],axis=1)
url_feature = url_feature[url_feature['label']!='未支付正常用户']

In [30]:
#分出训练集和测试集
X=url_feature.iloc[:,:-1]
Y=url_feature.iloc[:,-1]
test_size = 0.2
seed=7
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)


In [ ]:
#训练数据
#learning_rate ＝ 0.1 或更小，越小就需要多加入弱学习器；
#tree_depth ＝ 2～8；
#subsample ＝ 训练集的 30%～80%；
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from xgboost import plot_importance
import matplotlib.pyplot as plt
model = XGBClassifier()
model.fit(X_train, y_train)
learning_rate = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(learning_rate=learning_rate)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#画出特征重要性
#plot_importance(model)
#plt.show()

In [ ]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [62]:
#测试数据
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
target_names = ["恶拍用户","正常用户"]

y_train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("XGB的训练集情况如下:  %.2f%%" % (train_accuracy * 100.0))
print(classification_report(y_train, y_train_pred, target_names=target_names,digits=4))

y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("XGB的训练集情况如下:  %.2f%%" % (test_accuracy * 100.0))
print(classification_report(y_test, y_test_pred, target_names=target_names,digits=4))

XGB的训练集情况如下:  96.70%
              precision    recall  f1-score   support

        恶拍用户     0.8993    0.6525    0.7563      1862
        正常用户     0.9711    0.9938    0.9823     21868

   micro avg     0.9670    0.9670    0.9670     23730
   macro avg     0.9352    0.8232    0.8693     23730
weighted avg     0.9655    0.9670    0.9646     23730

XGB的训练集情况如下:  96.54%
              precision    recall  f1-score   support

        恶拍用户     0.8734    0.6259    0.7292       441
        正常用户     0.9706    0.9927    0.9815      5492

   micro avg     0.9654    0.9654    0.9654      5933
   macro avg     0.9220    0.8093    0.8554      5933
weighted avg     0.9634    0.9654    0.9628      5933

